In [13]:
# =======
# MODULES
# =======

import cv2
import time

In [14]:
# =========
# FONCTIONS
# =========

def SIFT(image):
    '''
    Entrée :
        - image : chemin vers l'image à analyser.
    Sortie :
        - points : liste des points clés détectés dans l'image.
        - descripteurs : descripteurs associés aux points clés.
    Description :
    Fonction utilisant l'algorithme SIFT pour détecter les points clés et calculer leurs descripteurs dans une image.
    '''
    # Lecture et redimensionnement de l'image
    img = cv2.imread(image, cv2.IMREAD_COLOR)
    img = redimensionner(img)

    # Conversion en niveaux de gris
    img_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Détection des points clés et calcul des descripteurs SIFT
    sift = cv2.SIFT_create()
    points, descripteurs = sift.detectAndCompute(img_gris, None)

    # Affichage des points clés sur l'image
    cv2.drawKeypoints(img, points, img, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Affichage de l'image avec les points clés
    cv2.imshow("Detecteur SIFT", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Retour des points clés et des descripteurs
    return points, descripteurs

def redimensionner(image):
    '''
    Entrée :
        - image : image à redimensionner.
    Sortie :
        - image redimensionnée.
    Description :
    Fonction qui redimensionne une image à la moitié de sa taille originale.
    '''
    hauteur, largeur = image.shape[:2]
    return cv2.resize(image, (largeur//2, hauteur//2), interpolation=cv2.INTER_AREA)

In [15]:
# =========
# PROGRAMME PRINCIPAL
# =========

# Liste des imagettes disponibles
lst_imagettes = [
    {"nom": "Aloy", "chemin": "Collection_images/Imagettes/Imagette_Aloy.png"},
    {"nom": "Kurumi Tokisaki", "chemin": "Collection_images/Imagettes/Imagette_Kurumi_Tokisaki.png"},
    {"nom": "Lelouch Vi Britannia", "chemin": "Collection_images/Imagettes/Imagette_Lelouch_Vi_Britannia.png"},
    {"nom": "Loona", "chemin": "Collection_images/Imagettes/Imagette_Loona.png"},
    {"nom": "Maomao", "chemin": "Collection_images/Imagettes/Imagette_Maomao.png"},
    {"nom": "March 7th", "chemin": "Collection_images/Imagettes/Imagette_March_7th.png"},
    {"nom": "Pikachu", "chemin": "Collection_images/Imagettes/Imagette_Pikachu.png"},
    {"nom": "Sage", "chemin": "Collection_images/Imagettes/Imagette_Sage.png"},
    {"nom": "Sonic", "chemin": "Collection_images/Imagettes/Imagette_Sonic.png"},
    {"nom": "Violet Evergarden", "chemin": "Collection_images/Imagettes/Imagette_Violet_Evergarden.png"}
    ]

choix = "o"

# ==================================
# BOUCLE PRINCIPALE DE L'APPLICATION
# ==================================

while choix.lower() == "o":
    # Affichage du menu et choix de l'imagette
    print("\n--- Bienvenue dans Character Finder ---\n")
    time.sleep(1)
    print("Cette application permet de détecter des personnages fictifs parmi une collection d'images.\n")
    time.sleep(2)
    print("-- Pour commencer, veuillez choisir une imagette à chercher :")
    time.sleep(1)

    # Affichage des options d'imagettes
    for i, imagette in enumerate(lst_imagettes):
        print(f"{i+1} - {lst_imagettes[i]["nom"]}")
    
    # Choix de l'imagette
    choix_imagette = int(input("Votre choix :"))
    print(f"\nVous avez choisi l'imagette de \"{lst_imagettes[choix_imagette-1]["nom"]}\".")
    time.sleep(1)

    imagette = lst_imagettes[choix_imagette-1]
    nom = imagette["nom"]
    
    # Affichage de l'imagette
    img = cv2.imread(imagette["chemin"], cv2.IMREAD_COLOR)
    img = redimensionner(img)
    cv2.imshow("Imagette choisie", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Extraction des points clés et des descripteurs de l'imagette
    points_imagette, descripteurs_imagette = SIFT(imagette["chemin"])

    # Initialisation du matcher de Brute Force
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    # Choix du dossier d'images à analyser
    print("\n-- Ensuite, veuillez choisir un dossier d'images :")
    time.sleep(1)

    # Affichage des options de dossiers
    for i, imagette in enumerate(lst_imagettes):
        print(f"{i+1} - {lst_imagettes[i]["nom"]}")

    # Choix du dossier
    choix_dossier = int(input("Votre choix :"))
    print(f"\nVous avez choisi le dossier \"{lst_imagettes[choix_dossier-1]["nom"]}\".")
    time.sleep(1)

    # Chemin du dossier choisi
    chemin = "Collection_images/" + lst_imagettes[choix_dossier-1]["nom"] + "/"

    # Nombre d'images présente dans le dossier
    nb_images = 1

    # Boucle permettant de parcourir toutes les images du dossier
    while True:
        # Récupère le chemin de l'image
        chemin_img = chemin + f"IMG_{nb_images}.jpg"
        img = cv2.imread(chemin_img, cv2.IMREAD_COLOR)

        # Fin du dossier si l'image n'existe pas
        if img is None:
            time.sleep(1)
            print(f"\nFin de l'analyse des images dans le dossier \"{nom}\".")
            time.sleep(1)
            break

        print(f"\n--- Analyse de l'image {nb_images} ---")
        time.sleep(1)

        # Extraction des descripteurs de l'image courante
        points_image, descripteurs_image = SIFT(chemin_img)

        # Matching KNN entre l'imagette et l'image courante
        pairs = bf.knnMatch(descripteurs_imagette, descripteurs_image, k=2)

        # Application du test de ratio de Lowe
        matches = []
        for m, n in pairs:
            if m.distance < 0.8 * n.distance:
                matches.append(m)

        print("Nombre de matches valides :", len(matches))
        time.sleep(1)

        # Résultat de la comparaison : personnage présent ou non
        SEUIL = 15
        if len(matches) > SEUIL:
            print("Personnage détecté")
            time.sleep(1)
        else:
            print("Personnage non détecté")
            time.sleep(1)

        # Passe à l'image suivante
        nb_images += 1

    # Relance ou arrête le programme
    print("\nSouhaitez-vous relancer l'application ? (o/n) : ", end="")
    reponse = input(" ")
    print(reponse)
    choix = reponse
    time.sleep(1)

    if reponse.lower() == "n":
        print("\nMerci d'avoir utilisé Character Finder ! Au revoir !")
    else:
        while reponse.lower() not in ["o", "n"]:
            print("\nRéponse invalide.")
            time.sleep(1)
            print("\nSouhaitez-vous relancer l'application ? (o/n) : ", end="")
            reponse = input(" ")
            print(reponse)
            choix = reponse
            time.sleep(1)


--- Bienvenue dans Character Finder ---

Cette application permet de détecter des personnages fictifs parmi une collection d'images.

-- Pour commencer, veuillez choisir une imagette à chercher :
1 - Aloy
2 - Kurumi Tokisaki
3 - Lelouch Vi Britannia
4 - Loona
5 - Maomao
6 - March 7th
7 - Pikachu
8 - Sage
9 - Sonic
10 - Violet Evergarden

Vous avez choisi l'imagette de "Aloy".

-- Ensuite, veuillez choisir un dossier d'images :
1 - Aloy
2 - Kurumi Tokisaki
3 - Lelouch Vi Britannia
4 - Loona
5 - Maomao
6 - March 7th
7 - Pikachu
8 - Sage
9 - Sonic
10 - Violet Evergarden

Vous avez choisi le dossier "Aloy".

--- Analyse de l'image 1 ---
Nombre de matches valides : 7
Personnage non détecté

--- Analyse de l'image 2 ---
Nombre de matches valides : 4
Personnage non détecté

--- Analyse de l'image 3 ---
Nombre de matches valides : 10
Personnage non détecté

--- Analyse de l'image 4 ---
Nombre de matches valides : 8
Personnage non détecté

--- Analyse de l'image 5 ---
Nombre de matches valides